In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.23.3
    Uninstalling mlxtend-0.23.3:
      Successfully uninstalled mlxtend-0.23.3


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [3]:
# load the data set ans show the first five transaction
data = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
data.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [17]:
purchased = data.unstack().unique()
print(purchased)

['Bread' 'Cheese' 'Meat' 'Eggs' 'Wine' 'Bagel' 'Pencil' 'Diaper' 'Milk'
 nan]


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [18]:
# Create an itemset based on unique products
items = {item: 0 for item in purchased}

# encoding the feature
for item in data.iloc[0]:
    if item in items:
        items[item] = 1

items

{'Bread': 1,
 'Cheese': 1,
 'Meat': 1,
 'Eggs': 1,
 'Wine': 1,
 'Bagel': 0,
 'Pencil': 1,
 'Diaper': 1,
 'Milk': 0,
 nan: 0}

In [24]:
reshaped_data = data.values.tolist()

# Flatten the list of items
flat_items = [item for sublist in reshaped_data for item in sublist]

# Reshape the data to a 2D array where each item is a row
flat_items_array = np.array(flat_items).reshape(-1, 1)

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
encoded_data = encoder.fit_transform(flat_items_array)

# Convert the encoded data to integer (1 and 0)
encoded_data = encoded_data.astype(int)

# Create a DataFrame with item names as columns
encoded_df = pd.DataFrame(encoded_data, columns=encoder.categories_[0])

# Now, create a customer ID list for the rows to map the one-hot encoding back to the original customers
customer_ids = []
for i, row in enumerate(reshaped_data):
    customer_ids.extend([i] * len(row))

# Add the customer IDs to the DataFrame
encoded_df['customer_id'] = customer_ids

# Pivot the DataFrame to get one-hot encoding by customer
final_df = encoded_df.groupby('customer_id').sum()

# create new dataframe from the encoded features
transformed_df = final_df

  # show the new dataframe
transformed_df.head()

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine,nan
customer_id,,,,,,,,,,
0,0,1,1,1,1,1,0,1,1,0
1,0,1,1,1,0,1,1,1,1,0
2,0,0,1,0,1,1,1,0,1,2
3,0,0,1,0,1,1,1,0,1,2
4,0,0,0,0,0,1,0,1,1,4


In [25]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
transformed_df = transformed_df.drop(columns='nan')
transformed_df.head()

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
customer_id,,,,,,,,,
0,0,1,1,1,1,1,0,1,1
1,0,1,1,1,0,1,1,1,1
2,0,0,1,0,1,1,1,0,1
3,0,0,1,0,1,1,1,0,1
4,0,0,0,0,0,1,0,1,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [29]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(transformed_df, min_support=0.2, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.504762,(Bread)
2,0.501587,(Cheese)
3,0.406349,(Diaper)
4,0.438095,(Eggs)
5,0.476190,(Meat)
6,0.501587,(Milk)
7,0.361905,(Pencil)
8,0.438095,(Wine)
9,0.279365,"(Bagel, Bread)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [30]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules.drop(columns=['zhangs_metric'], inplace=True)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
2,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
3,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
4,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
5,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
8,"(Meat, Eggs)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667
9,"(Meat, Cheese)",(Eggs),0.323810,0.438095,0.215873,0.666667,1.521739,0.074014,1.685714


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

The metrics in the dataset describe various aspects of association rules and their strength. Antecedent support represents how often the items in the rule’s antecedent appear in the transactions. For instance, (Bagel) has an antecedent support of 42.54%, meaning “Bagel” appears in 42.54% of all transactions. Consequent support indicates how often the items in the rule’s consequent occur. For example, (Bread) has a consequent support of 50.48%, showing that “Bread” appears in half of the transactions. Support measures how frequently both the antecedent and consequent appear together; for instance, (Bagel) -> (Bread) has a support of 27.94%, meaning these items are bought together in nearly 28% of all transactions.

Confidence evaluates the likelihood of the consequent being purchased when the antecedent is purchased, reflecting the strength of the association. For example, (Bagel) -> (Bread) has a confidence of 65.67%, indicating that about two-thirds of the time “Bagel” is purchased, “Bread” is also bought. Lift goes further by comparing this likelihood to random chance, with a value greater than one suggesting a positive association. The lift of 1.3 for (Bagel) -> (Bread) means that “Bread” is 1.3 times more likely to be purchased with “Bagel” than by random chance. Leverage quantifies the difference between observed and expected co-occurrence of items, with positive values indicating some association. For instance, (Bagel) -> (Bread) has a leverage of 0.064, showing a small but notable association. Conviction, on the other hand, measures the reliability of a rule, with higher values indicating a lower likelihood of failure. For example, (Bagel) -> (Bread) has a conviction of 1.44, meaning this rule is 1.44 times more reliable than random prediction.

From the dataset, certain rules exhibit stronger associations. For example, (Meat, Eggs) -> (Cheese) has the highest lift value (1.61), showing that “Cheese” is very likely to be purchased when “Meat” and “Eggs” are bought together. Similarly, (Meat, Milk) -> (Cheese) demonstrates the highest confidence (83.12%), meaning most transactions involving “Meat” and “Milk” also include “Cheese.” Conversely, rules like (Wine) -> (Cheese) exhibit weaker associations, with lower lift (1.22) and leverage values, suggesting less impactful relationships. The conviction of 2.61 for (Meat, Eggs) -> (Cheese) highlights its strong predictive power, making it a reliable rule for forecasting customer behavior.

In summary, the dataset reveals that “Meat,” “Eggs,” and “Cheese” have strong associations and should be prioritized for marketing strategies such as product bundling or placement. Weaker associations, such as those involving “Wine,” might still offer insights for secondary marketing efforts. These findings provide actionable insights for improving customer engagement and sales through data-driven decision-making.


**References:**

-https://towardsdatascience.com/association-rules-2-aa9a77241654

-https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/